In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np # linear algebra
import cv2

import os
import random
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, 
    Dense, Dropout, 
    Flatten)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(os.listdir("/content/gdrive/My Drive/breast-20200512T051500Z-001/breast_data"))

['normal', 'cancer']


In [ ]:
IMG_SIZE = 128

In [ ]:
CATEGORIES = ['cancer', 'normal']
dataset = []

def generate_data():
    for category in CATEGORIES:
        path = f'/content/gdrive/My Drive/breast-20200512T051500Z-001/breast_data/{category}'
        class_id = CATEGORIES.index(category)
        for image in os.listdir(path):
            try:
                image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_COLOR)
                image_array = cv2.resize(image_array, (IMG_SIZE , IMG_SIZE))
                dataset.append([image_array, class_id])
            except Exception as e:
                print(e)
    random.shuffle(dataset)
                
generate_data()

In [ ]:
data = []
labels = []
for features, label in dataset:
    data.append(features)
    labels.append(label)

In [ ]:
print(len(dataset))

In [ ]:
data = []
labels = []
for features, label in dataset:
    data.append(features)
    labels.append(label)

In [ ]:
data = np.array(data)
data.reshape(-1, 128, 128, 3)

In [ ]:
pickle.dump(data, open("data1.pickle", "wb"))

pickle.dump(labels, open("labels1.pickle", "wb"))

In [ ]:
data = pickle.load(open("data1.pickle", "rb"))

labels = pickle.load(open("labels1.pickle", "rb"))

In [ ]:
train_data, data, train_labels, labels = train_test_split(data, 
                                                          labels,
                                                          test_size=0.15)

test_data, validation_data, test_labels, validation_labels = train_test_split(data, 
                                                                    labels,
                                                                    test_size=0.75)

In [ ]:
plt.figure(figsize=(10, 10))
i = 0
for i in range(len(test_data)):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(test_data[i])
    if(test_labels[i] == 0):
        plt.xlabel('Infected')
    else:
        plt.xlabel('Uninfected')
    i += 1
plt.show()

In [ ]:
datagen_train = ImageDataGenerator(rescale=1./255,
                            rotation_range=45,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

datagen_test = ImageDataGenerator(rescale=1./255)
datagen_validation = ImageDataGenerator(rescale=1./255)

In [ ]:
train_d=datagen_train.fit(train_data)
datagen_test.fit(test_data)
vod_d=datagen_test.fit(validation_data)

In [ ]:
model = Sequential([
      
       Flatten(),
       Dense(units=64,kernel_initializer = 'uniform', activation = 'relu',input_shape=(IMG_SIZE,IMG_SIZE,3)) ,
       #Dense(units =64, kernel_initializer = 'uniform', activation = 'relu'),
       Dense(units =32, kernel_initializer = 'uniform', activation = 'relu'),
       #Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'),
       Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid')
])

In [ ]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)
#model.compile(loss='categorical_crossentropy', optimizer=opt,metrics = ['accuracy'])
#model.compile(optimizer =opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
# Fitting the ANN to the Training set
#history=model.fit(train_data,y=np.array(train_labels),batch_size = 8, epochs = 100,verbose=1,validation_data=(test_data, np.array(test_labels)))
BATCH_SIZE = 32
epochs = 30
#history=model.fit(train_data, y=np.array(train_labels), batch_size=BATCH_SIZE,epochs=epochs,verbose=1,validation_data=(test_data, np.array(test_labels)))
BATCH_SIZE = 32
epochs = 30
#history=model.fit(train_data, y=np.array(train_labels), batch_size=BATCH_SIZE,epochs=epochs,verbose=1,validation_data=(test_data, np.array(test_labels)))
#history=model.fit(train_data,y=np.array(train_labels), batch_size = 10, epochs = 100)

In [ ]:
accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

print(f'Training Accuracy: {np.max(accuracy)}')
print(f'Training Loss: {np.min(loss)}')
print(f'Validation Accuracy: {np.max(val_accuracy)}')
print(f'Validation Loss: {np.min(val_loss)}')

In [ ]:
epochs_range = range(100)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.xlabel('epochs')
plt.ylabel('Accuracy rate')
plt.plot(epochs_range, accuracy, label="Training Accuracy")
plt.plot(epochs_range, val_accuracy, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.xlabel('epochs')
plt.ylabel('loss rate')
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
class_names = ['cancer', 'normal']
def plot_images(i, predictions_array, true_labels, images):
    predictions_array, true_label, img = predictions_array[i], true_labels[i],images[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    plt.imshow(img)
    
    predicted_label = np.argmax(predictions_array)
        
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                        100*np.max(predictions_array),
                                        class_names[true_label]))

In [ ]:
random.shuffle(test_data)
predictions = model.predict(test_data)


In [ ]:
num_rows = 8
num_cols = 6
num_images = num_rows * num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(len(test_data)):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_images(i, predictions, test_labels, test_data)

In [ ]:
model.summary( )

In [ ]:
loss , accuracy = model.evaluate( test_data , np.array(test_labels) )
print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ) )

In [ ]:
pred=model.predict( test_data).argmax( axis=1 ) 
print ( pred )

In [ ]:
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
print('F1_score : ',f1_score(test_labels,pred))
print('sensitibity : ',metrics.recall_score(test_labels, pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, pred))

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef

fpr , tpr , thresholds = roc_curve (test_labels,pred)
plot_roc_curve (fpr,tpr)
auc_score=roc_auc_score(test_labels,pred) 
print(auc_score) 
print(matthews_corrcoef(test_labels,pred))

In [ ]:
def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show() 
   

In [ ]:
confusion = confusion_matrix(test_set.classes, y_pred)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('sensitivity =',TP/(TP+FN))
print(metrics.recall_score(test_set.classes, y_pred))
recall=metrics.recall_score(test_set.classes, y_pred)
print('specificity =',TN/(TN+FP))
print('F1_score = ',f1_score(test_set.classes,y_pred))
#print('F1_score = ',2*((metrics.precision_score(test_set.classes, y_pred)*recall)/(metrics.precision_score(test_set.classes, y_pred)+recall)))

In [ ]:
history.history

In [ ]:
classifier.metrics_names

In [ ]:
import matplotlib.pyplot as plt



loss = history.history['loss']
val_loss = history.history['val_loss']


acc = history.history['acc']
val_acc = history.history['val_acc']



epochs = range(len(loss))

plt.figure()
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation loss')
plt.legend()

plt.show()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()